In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

import plotly.express as px


import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

import time

2.6.0+cu126
12.6
90501
CUDA available: True
GPU: NVIDIA GeForce RTX 4070 SUPER


In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Path to the .env file inside 01 - Documentation
env_path = Path("01 - Documentation") / "keys.env"

# Load the .env file from that path
load_dotenv(dotenv_path=env_path)

# Now you can access your keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

print(API_KEY[:4] + "****")  # optional: verify it's working

PKD5****


In [9]:
myAssets = ["AMZN", "META", "GOOGL", "AMD", "NKE", "UBER", "COST", "JPM", "CRM", "TXRH"]
shareCount = [15, 4.73, 14.43, 25.21, 30.16, 20, 1.25, 4.84, 4.36, 5.68]

client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

end_date = datetime.today()
start_date = end_date - timedelta(days=365)  # 1 year of daily prices

request = StockBarsRequest(
    symbol_or_symbols=myAssets,
    timeframe=TimeFrame.Day,
    start=start_date,
    end=end_date
)

bars = client.get_stock_bars(request).df

# Get the last row (most recent date) per asset
latest_closes = bars['close'].groupby(level=0).last()
prices = latest_closes.to_dict()

print(prices)


print(bars.head())
print(bars.tail())
print(bars.columns)


{'AMD': 87.57, 'AMZN': 173.305, 'COST': 990.0, 'CRM': 249.74, 'GOOGL': 151.315, 'JPM': 233.72, 'META': 502.5888, 'NKE': 55.07, 'TXRH': 162.0, 'UBER': 75.485}
                                    open    high       low   close     volume  \
symbol timestamp                                                                
COST   2024-04-18 04:00:00+00:00  715.19  718.95  709.5700  711.25  1846393.0   
       2024-04-19 04:00:00+00:00  713.03  714.87  702.0000  709.51  2190356.0   
       2024-04-22 04:00:00+00:00  714.86  718.26  706.1700  715.46  1437110.0   
       2024-04-23 04:00:00+00:00  719.03  724.27  716.2975  722.68  1176191.0   
       2024-04-24 04:00:00+00:00  719.71  725.43  716.0900  723.89  1206638.0   

                                  trade_count        vwap  
symbol timestamp                                           
COST   2024-04-18 04:00:00+00:00      48811.0  713.438859  
       2024-04-19 04:00:00+00:00      63814.0  708.401716  
       2024-04-22 04:00:00+00:00  

In [4]:
# Calculate the value of each position
position_values = [shareCount[i] * prices[symbol] for i, symbol in enumerate(myAssets)]

# Create a DataFrame for visualization
Assetsdf = pd.DataFrame({
    "Symbol": myAssets,
    "Shares": shareCount,
    "Price": [prices[symbol] for symbol in myAssets],
    "Value": position_values
})

Assetsdf

,Symbol,Shares,Price,Value
0,AMZN,15.00,173.9400,2609.10000
1,META,4.73,504.2900,2385.29170
2,GOOGL,14.43,151.6100,2187.73230
3,AMD,25.21,87.4650,2204.99265
4,NKE,30.16,54.9700,1657.89520
5,UBER,20.00,75.5400,1510.80000
6,COST,1.25,988.2394,1235.29925
7,JPM,4.84,233.8350,1131.76140
8,CRM,4.36,250.1700,1090.74120
9,TXRH,5.68,162.0250,920.30200


In [ ]:
portfolioValue = sum(Assetsdf["Value"])
print("Portfolio Value: $", portfolioValue)

fig = px.pie(Assetsdf,
             names='Symbol',
             values='Value',
             title='Portfolio Allocation by Market Value: $' + str(portfolioValue),
             hole=0,  # Set to >0 if you want a donut chart instead
             width=800,
             height=500)

fig.update_traces(textinfo='label+percent', textposition='inside')

fig.show()


Portfolio Value: $ 16933.9157


In [6]:
import plotly.graph_objects as go

close_prices = bars.reset_index().pivot(index="timestamp", columns="symbol", values="close").sort_index()

normalized_prices = close_prices / close_prices.iloc[0]

fig = go.Figure()

for symbol in myAssets:
    fig.add_trace(go.Scatter(
        x=normalized_prices.index,
        y=normalized_prices[symbol],
        mode='lines',
        name=symbol
    ))

fig.update_layout(
    title='Normalized Portfolio Stock Prices (Base = 1)',
    xaxis_title='Date',
    yaxis_title='Normalized Price',
    width=800,
    height=600
)

fig.show()


### Sharpe Ratio

The **Sharpe Ratio** measures the risk-adjusted return of a portfolio or investment. It indicates how much **excess return** you earn for each unit of **volatility** (risk).

It is defined as:

$$
\text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p}
$$

Where:

- $R_p$ = Portfolio return (mean)  
- $R_f$ = Risk-free rate of return  
- $\sigma_p$ = Standard deviation of the portfolio returns  

A higher Sharpe ratio indicates a more favorable risk-adjusted performance.


In [7]:
# Pivot to get multi-asset wide format (Date x Symbols)
returns = close_prices.pct_change().dropna()

# Get normalized weights
initial_prices = close_prices.iloc[0]
weights = pd.Series(shareCount, index=myAssets)
weights = weights / (weights * initial_prices).sum()

# Portfolio daily returns
portfolio_returns = returns.dot(weights)

risk_free_rate = 0.043070  # 4% annual
trading_days = 252

# Convert Rf to daily rate
daily_rf = risk_free_rate / trading_days

excess_returns = portfolio_returns - daily_rf

sharpe_ratio = (excess_returns.mean() / excess_returns.std()) * np.sqrt(trading_days)
print(f"📊 Sharpe Ratio: {sharpe_ratio:.2f}")
print("Portfolio mean daily return:", portfolio_returns.mean())
print("Portfolio daily std dev:", portfolio_returns.std())
print("Risk-free daily:", daily_rf)
print("Excess return mean:", excess_returns.mean())
print("Sharpe Ratio:", sharpe_ratio)



📊 Sharpe Ratio: -24.31
Portfolio mean daily return: -3.872026521717886e-06
Portfolio daily std dev: 0.0001141232290468749
Risk-free daily: 0.0001709126984126984
Excess return mean: -0.0001747847249344163
Sharpe Ratio: -24.31250424710757


In [8]:
cumulative_returns = (1 + portfolio_returns).cumprod()

fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, name='Portfolio Value'))
fig.update_layout(title='Cumulative Portfolio Value (Normalized)',
                  xaxis_title='Date', yaxis_title='Growth Factor')
fig.show()
